Scratchwork for updating metadata fields to reflect newest version of experiment and harmonize import into database. 

* Using AWS and rclone to sync **FROM:s3 TO:local** (using exclude file):
    * `rclone sync antenna-aws:antenna-fs ~/data/antenna-fs -P --exclude-from sync_exclude.txt --dry-run`
* Sync **FROM:local TO:s3** using an exclude file (using exclude file):
    * `rclone sync ~/data/antenna-fs antenna-aws:antenna-fs -P --exclude-from sync_exclude.txt --dry-run`
* remove `--dry-run` after testing

In [ ]:
import json
import tqdm
from itertools import chain
from pathlib import Path

In [ ]:
%pdb on

Add `probe_angle` field to metadata since it may now change with a new platform/holder.

In [ ]:
data_path = Path('~/data/antenna-fs/').expanduser()
session_json_paths = data_path.glob('*/exp_ephys_*/*/session.json')

for session_json_path in tqdm.tqdm(session_json_paths):
    session_meta = json.load(session_json_path.open())
    if not 'probe_angle' in session_meta.keys():
        session_meta.update({'probe_angle': 15})
        with open(session_json_path.as_posix(), 'w') as file:
            json.dump(session_meta, file, indent=4) # matches formatting from matlab

Rename `params_hash_piezo` to `stim_hash_piezo` in all files where it is required

In [ ]:
data_path = Path('~/data/antenna-fs/').expanduser().joinpath('raw')
trial_json_paths = data_path.glob('exp_ephys_*/session_*/*/*.json')

for trial_json_path in tqdm.tqdm(trial_json_paths):
    # rsync/osx garbage
    if trial_json_path.parts[-1][0] != '.':
        trial_meta = json.load(open(trial_json_path))
        if not 'stim_hash_piezo' in trial_meta.keys():
            trial_meta.update({'stim_hash_piezo': trial_meta['params_hash_piezo']})
            trial_meta.pop('params_hash_piezo')

            with open(trial_json_path.as_posix(), 'w') as file:
                json.dump(trial_meta, file, indent=4) # matches formatting from matlab

Add `antenna_side` because (yay!) I can now choose based on driver line. All old experiments in this dataset (so, NOT suction electrode experiments) use the _fly's_ right side. 

In [ ]:
data_path = Path('~/data/antenna-fs/').expanduser()
session_json_paths = data_path.glob('*/exp_ephys_*/*/session.json')

for session_json_path in tqdm.tqdm(session_json_paths):
    session_meta = json.load(session_json_path.open())
    if not 'antenna_side' in session_meta.keys():
        session_meta.update({'antenna_side': 'right'})
        with open(session_json_path.as_posix(), 'w') as file:
            json.dump(session_meta, file, indent=4) # matches formatting from matlab